## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [1]:
import numpy as np
import pandas as pd
from numba import jit

In [3]:
% load_ext line_profiler

In [2]:
arr_A = np.random.randint(0, 1000, size=1_000_000)

array([389, 698, 989, ..., 402, 415, 120])

In [5]:
% % timeit

sum(arr_A) / len(arr_A) + 100

61.6 ms ± 1.67 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
% % timeit

np.mean(arr_A) + 100

1.11 ms ± 197 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
@jit(nopython=True)
def numba_speed_up(arr):
    return sum(arr) / len(arr) + 100


% timeit numba_speed_up(arr_A)

535 µs ± 56.1 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [10]:
from string import ascii_lowercase

n = 2_000_000
df = pd.DataFrame(np.random.random(size=(n, 4)), columns=[f'random_{i}' for i in range(4)])
df['key'] = np.random.default_rng().choice(list(ascii_lowercase), size=n)
df

,random_0,random_1,random_2,random_3,key
0,0.210958,0.977723,0.568511,0.072595,c
1,0.086726,0.069594,0.756308,0.435492,z
2,0.657178,0.146990,0.852084,0.193332,l
3,0.088088,0.339943,0.851313,0.301842,j
4,0.199910,0.736553,0.749312,0.954224,k
...,...,...,...,...,...
1999995,0.352835,0.424604,0.706668,0.930801,h
1999996,0.091110,0.246090,0.379155,0.105727,a
1999997,0.480427,0.724575,0.547280,0.221321,n
1999998,0.386002,0.784489,0.580400,0.926004,e


In [11]:
def bad_solution(df_):
    letters = ['a', 'b', 'c', 'd', 'e']
    dfs = []
    for letter in letters:
        q = df_[df_['key'] == letter]
        dfs.append(q)
    return pd.concat(dfs, axis=0)


%timeit bad_solution(df)

563 ms ± 31.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
def inline_solution(df_):
    return df_[df_['key'].isin({'a', 'b', 'c', 'd', 'e'})]


%timeit inline_solution(df)

89.4 ms ± 1.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
def for_profiling_solution(df_):
    letters = {'a', 'b', 'c', 'd', 'e'}
    mask = df_['key'].isin(letters)
    return df_[mask]


%lprun -f for_profiling_solution for_profiling_solution(df)

## Лабораторная работа 3

In [ ]:
# !pip install line_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [10]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)

    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


#### [версия 2]
* Уточнены формулировки задач 1, 3, 4